In [2]:
source("utils/plot.R")

In [3]:
data <- read_tsv(snakemake@input[[1]], col_types=cols()) %>%
    mutate(yDodge=rep(rep(c(1, -1), length.out=21), 21),
           WindowNumber=as.numeric(factor(WindowSize)),
           yInPlot=WindowNumber*1.5 + yDodge*0.25,
          ) %>%
    group_by(WindowSize) %>%
    mutate(Covering=(max(CenterPosition) + WindowSize/2) - (min(CenterPosition) - WindowSize/2))

In [4]:
windows_wanted <- quantile(seq(1, 21), probs=c(0, 0.25, 0.5, 0.75, 1.0))

my_window_sizes <- data %>%
    select(WindowSize, Covering, WindowNumber, yInPlot) %>%
    distinct() %>%
    filter(WindowNumber %in% windows_wanted) %>%
    group_by(WindowSize, Covering, WindowNumber) %>%
    summarize(Break=mean(yInPlot)) %>%
    mutate(
        CoveringLabel=recode(Covering,
                            `11000`='11kb',
                            `33968`='34kb',
                            `104885`='105kb',
                            `323851`='324kb',
                            `999999`='1Mb'),
    )

In [5]:
my_window_sizes

In [6]:
make_kb_label <- function(br) {
    paste0(br, 'kb')
}

In [7]:
chosen_ws_to_annotate <- 131406

df <- data %>%
    mutate(
        ToAddText = Covering == chosen_ws_to_annotate
    )

annotpos_df <- df %>%
    filter(Covering == chosen_ws_to_annotate) %>%
    slice_max(CenterPosition)
annotation_position <- annotpos_df[['CenterPosition']] + annotpos_df[['WindowSize']]

annotation <- tibble(
    x = annotation_position,
    y = annotpos_df[['yInPlot']],
    label = paste0(
        "Subwindows of size ", round(annotpos_df[['WindowSize']]/1000), "kb,\ncovering ", round(annotpos_df[['Covering']]/1000), "kb"
    )
)

In [8]:
fig <- ggplot(df) +
    geom_vline(xintercept=0, colour='darkred') +
    geom_tile(aes(x=CenterPosition/1000 - 500,
                  y=yInPlot,
                  width=WindowSize/1000,
                  height=0.5,
                  fill=ToAddText),
              colour='black') +
    geom_text(data=annotation, aes(x=x/1000 - 500, y=y, label=label),
             hjust=0, nudge_y=-0.5, nudge_x=15, colour='darkred', size=2.5, family="Roboto Condensed") +
    scale_y_continuous(breaks=my_window_sizes$Break, labels=my_window_sizes$CoveringLabel) +
    scale_x_continuous(labels=make_kb_label) +
    scale_fill_manual(values=c('lightgrey', 'darkred')) +
    labs(
        x = 'Relative sweep position',
        y = 'Total subwindow coverage'
    ) +
    sweeps_theme +
    theme(
        panel.grid.minor = element_blank(),
        legend.position = 'none'
    )

In [15]:
sweeps_save(snakemake@output[[1]], fig, asp=1.618)